### Prérequis: Avoir fait le tp Transformer.ipynb

# Génération de texte
Félicitation d'avoir fini le tp sur les transformers (ou pas). Ici, on va s'amuser à générer des histoires avec un transformer (ou presque).

N'oubliez pas de mettre le runtime en GPU si vous êtes sur colab.

## Idée
Je ne sais pas si vous avez remarqué, mais pour de la génération de texte, il n'y pas de phrase à encoder, contrairement à la traduction de texte où la phrase à encoder est la phrase à traduire.

On va donc utiliser juste la partie decodeur du transformer. Mais si vous vous souvenez, il y a la couche Cross Attention dans la partie decodeur, qui n'est pas nécessaire ici car on n'a pas de phrase à encoder. On va donc utiliser un decoduer sans la Cross Attention.

![decoderonly](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/decoderonly.png)

Bref, l'idée est la suivante: on va entraîner notre modèle à compléter des phrases.

Exemple:

Entrée: ["\<start>", "Je", "suis", "un", "chat", "<end>"]

Sortie: ["Je", "suis", "un", "chat", "\<end>"]

Le modèle va alors:
- Prédire "Je" à partir de "\<start>"
- Prédire "suis" à partir de "\<start> Je"
- Prédire "un" à partir de "\<start> Je suis"
- etc...

Puis, pendant l'inference, on va juste lui donner "\<start>" et il va compléter la phrase jusqu'à "\<end>".

Si vous vous rappelez, la sortie de notre modèle est des vecteurs de probabilité et dans le tp précédent, on a utilisé l'argmax de ces vecteurs de probabilité pour avoir le mot prédit.

Mais si ici on prend l'argmax du vecteur de probabilité à chaque fois, on va avoir juste avoir la même phrase à chaque fois qu'on génère du texte.

Donc on va utiliser un autre moyen pour générer du texte: le sampling.

Le sampling consiste à prendre un mot en fonction de sa probabilité. Par exemple, si le mot "chat" a une probabilité de 0.8, on va le prendre 80% du temps.

Donc avec les vecteurs de probabilité que notre modèle nous donne, on va prendre un mot en fonction de sa probabilité.

Avant de sampler, on va diviser les vecteurs (avant le softmax) par un facteur qu'on appelle la température. La température est un hyperparamètre qui va déterminer la diversité du texte généré. Plus la température est grande, plus le texte généré sera diversifié. Plus la température est petite, plus le texte généré sera similaire à l'entrée, mais plus robuste à l'erreur, car avec une température trop grande, les probabilités des mots vont être trop proches et on risque d'avoir des mots qui n'ont pas de sens.

Remarque: C'est exactement ce que fait ChatGPT. Il est aussi un decoder-only transformer mais il a 175 milliards de paramètres, un peu plus gros que le modèle qu'on va faire ici.

## Préparation des données
On va utiliser le dataset WritingPrompts. C'est un dataset qui contient les prompts et les histoires associées.

Pour l'instant, on va utiliser que les histoires pour l'entraînement.

In [ ]:
import requests

url = "https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/colab_train.txt"
r = requests.get(url)
dataset = r.text

# save file
with open("colab_train.txt", "w") as f:
    f.write(dataset)

`dataset` est une string qui contient toutes les histoires. Chaque ligne correspond à une histoire.
`<newline>` est un token qui correspond à un retour à la ligne.


In [ ]:
print(dataset.split("\n")[0])  # On affiche la première histoire

## Tokenization
Ici, on va utiliser la tokenization Byte-Pair Encoding (BPE). C'est une tokenization qui va découper les mots en sous-mots. Par exemple, "chat" peut être découpé en "cha" et "t". C'est une tokenization qui est très utilisée dans les modèles de génération de texte, par exemple ChatGPT.
La tokenization mot par mot est aussi possible, mais elle n'est pas efficace. Par exemple, pour notre modèle (avant l'entraînement), "eat" et "eating" sont deux mots différents. Donc si on utilise une tokenization mot par mot, on va avoir deux embeddings différents pour "eat" et "eating", alors qu'ils devraient être liés en quelque sorte.

### Principes du BPE
La première question qu'on se pose est alors: Comment découper les mots en sous-mots?

On va découper les mots en sous-mots en fonction de la fréquence des sous-mots. Par exemple, si "chat" et "chien" sont deux mots très fréquents, alors "ch" va être un sous-mot très fréquent.

On part d'un vocabulaire de base constitué de caractères individuels.
Au début, on peut considérer chaque mot comme une séquence de caractères: "chat" = "c", "h", "a", "t".

Ensuite, on compte la fréquence de chaque paire de caractères (ou bigrammes) adjacents dans le corpus. La paire de caractères la plus fréquente est ensuite fusionnée pour former un nouveau token du vocabulaire. Par exemple, si "ch" est la paire de caractères la plus fréquente, on va fusionner "c" et "h" pour former un nouveau token "ch".

On répète ce processus plusieurs fois, en fusionnant à chaque fois les paires de tokens les plus fréquentes pour former de nouveaux tokens. À chaque itération, le vocabulaire augmente avec l'ajout de nouveaux sous-mots.

Remarque alors la tokenization BPE dépend du dataset.
Il faut donc "entraîner" la tokenization BPE sur notre dataset.

### Implémentation

On ne va pas l'implémenter car ce n'est pas le but de ce TP, on va donc utiliser la librarie sentencepiece qui contient déjà le BPE.
Il faut installer sentencepiece avec `pip install sentencepiece`.
Si vous êtes sur colab: `!pip install sentencepiece`

#### Entraînement
On va entraîner la tokenization BPE sur notre dataset et on choisit d'avoir 2500 tokens au maximum.
Donc notre tokenization BPE contiendra les 2500 sous-mots les plus fréquents.

In [ ]:
from sentencepiece import SentencePieceTrainer

input_file = 'colab_train.txt'
max_num_words = 2500
model_type = 'bpe'
model_prefix = 'sentencepiece'
pad_id = 0
unk_id = 1
start_id = 2
end_id = 3

sentencepiece_params = ' '.join([
	'--input={}'.format(input_file),
	'--model_type={}'.format(model_type),
	'--model_prefix={}'.format(model_prefix),
	'--vocab_size={}'.format(max_num_words),
	'--pad_id={}'.format(pad_id),
	'--unk_id={}'.format(unk_id),
	'--bos_id={}'.format(start_id),
	'--eos_id={}'.format(end_id)
])

print(sentencepiece_params)
SentencePieceTrainer.train(sentencepiece_params)

#### Utilisation
Tout d'abord, on va load le modèle qu'on vient d'entraîner.

In [ ]:
from sentencepiece import SentencePieceProcessor

sp = SentencePieceProcessor()
sp.load("{}.model".format(model_prefix))

Pour tokeniser une phrase, on utilise la fonction `encode_as_pieces`.

In [ ]:
sp.encode_as_pieces("Je suis un chat")

Pour décoder une liste de tokens, on utilise la fonction `decode_pieces`.

In [ ]:
tokenised = sp.encode_as_pieces("Je suis un chat")
sp.decode_pieces(tokenised)

On peut aussi encoder une phrase en entier avec la fonction `encode_as_ids`. Ça évite d'utiliser vocab de `torchtext`.

In [ ]:
sp.encode_as_ids("Je suis un chat")

Pour décoder une liste d'ids, on utilise la fonction `decode_ids`.

In [ ]:
input_ids = sp.encode_as_ids("Je suis un chat")
sp.decode_ids(input_ids)

Pour obtenir la taille du vocabulaire, on utilise la fonction `get_piece_size`.

In [ ]:
sp.get_piece_size()

## Pytorch-Lightning
Pour ce TP, on va utiliser pytorch lightning pour nous simplifier la vie. On va utiliser la précision '16-mixed' afin de réduire la mémoire utilisée par le modèle et donc d'augmenter la taille du modèle, et on va éventuellement utiliser un learning rate scheduler.

Pour installer pytorch lightning: `pip install pytorch-lightning`.
Si vous êtes sur colab: `!pip install pytorch-lightning`

### Exercice: Preprocessing les données et créer le dataloader

Il faut maintenant tokeniser les données et les transformer en liste d'entiers, en utilisant la tokenization BPE qu'on vient d'entraîner.

Comme la tokenisation BPE résulte en des listes de tokens de tailles assez longues, il faut les tronquer sinon on va avoir des problèmes de mémoire.
On va donc définir `max_len = 1500` et tronquer les listes de tokens à cette taille.
Remarque: vous pouvez réduire `max_len` si vous voulez augmenter la taille du modèle ou la batch size.


Il faut aussi padder les données pour qu'elles aient la même taille.
Dans les TP précédents, vous avez padder à la main, ce qui est assez long. Vous pouvez utiliser `torch.nn.utils.rnn.pad_sequence` pour padder les données.
La documentation est ici: https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html

Il faut aussi créer un dataloader avec `torch.utils.data.DataLoader`, en utilisant `torch.utils.data.TensorDataset` pour créer le dataset qui contient les données tokenisées et paddées, et les masks éventuels.
Exemple de dataloader:
```python
train_dataset = torch.utils.data.TensorDataset(x_train, key_padding_mask, loss_mask)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=3)
```

In [ ]:
...
train_loader = ...

### Exercice: Création du modèle

Implémenter votre transformer-decoder-only ici.

In [ ]:
import torch
import torch.nn as nn

...

### Exercice: Création du modèle lightning
Ici, on va créer le modèle pytorch lightning. Il faut donc créer la fonction `forward` et la fonction `training_step`.
Remarque: vous pouvez commencer par un modèle de taille: `d_model = 256`, `n_heads = 8`, `n_layers = 5`.
Normalement vous devriez pas avoir de problème de mémoire avec `max_len = 1500` et `batch_size = 16`.

In [ ]:
import pytorch_lightning as pl

class MyModel(pl.LightningModule):
	def __init__(self):
		super().__init__()
		self.model = ...

	def training_step(self, batch, batch_idx):
		...

	def configure_optimizers(self):
		...

model = MyModel()

checkpoint_callback = pl.callbacks.ModelCheckpoint(
...
)
trainer = pl.Trainer(max_epochs=1000, precision='16-mixed', callbacks=[checkpoint_callback])
trainer.fit(model, train_loader)

Vous pouvez ajouter ce code dans la fonction `training_step` afin de voir l'output de votre modèle pendant l'entraînement.
```python
if batch_idx == 0:
	self.model.eval()
	cur_sentence = [bos_id, sp.encode_as_ids("I")[0]]

	with torch.no_grad():
		for i in range(max_len):
			cur_input = torch.tensor(cur_sentence).unsqueeze(0).to(self.device)
			output = model(cur_input)
			output = output[:, -1, :]
			output = output[0].argmax()
			cur_sentence.append(output.item())
			if output.item() == eos_id:
				break
	print(sp.decode_ids(cur_sentence))

	self.model.train()
```

## Génération de texte
Il faut maintenant générer du texte avec notre modèle, en utilisant le sampling.

Après 30 minutes d'entraînement sur colab, j'ai obtenu:

"I'm so hungry i can't tell people i have the job. Most of the couple things you couldn't pay more than a damn job. I can have a job of money. So i also try to deal with keeping him to make some work. He's a good deal. You know him. He doesn't eaten things sometimes. He thinks he doesn't go crazy is either crazy i'd be a bad shitty writer. You sure as hell knows, but he does n't buyback. I'm gon na make it out to my friend when i get him a lot more excuse for me. I don't care if he can, but i don't really help. I need him to school. He's too good for me. I don't need to be able to go home. Don't care to leave. Don't want to take a drink, i can't leave. Don't look at me like i need him too much to hide in the day. Don't want to go away. Don't drink. Don't drink. Don't drink. Don't listen and take a penis to the cash and then when all i can't even get it. Don't take me on my own phone. Don't we leave to webbing together and started talking to ourselves. Don't you believe me ? maybe i don't want that my friend. I don't want to say something. I don't expect you. I always thought that man is just like me. I don't want to scream when heates. I always wanted to be much better than friendly. I always wanted to be the guy. But not me. I'm never worried about what the fuck people say. I always wanted to talk to you. I always thought about it was just the worst. You know that i'm crazy because i'm pretty sure i'm not saying that because i'll never worried about you grow up and i have a friendship on me. I can't stay there. I've been trying to get away with people. I never hurt anyone but i've been with for so long after i was doomed to ever have the best idea of my life and i'll ever be happy about this guy. I've missed his name, i've got to have the money. I don't care for the way that matter. I always say goodbye. I'll never good enough for the way things i'll ever get to know you. I don't want to know why you grow up with someone else to do with you with your friend."

### Amélioration possible
Vous pouvez essayer d'augmenter la taille de votre modèle, par exemple augmenter `d_model` ou `n_layers`, mais il faudra réduire la batch_size ou `max_len` sinon vous allez avoir des problèmes de mémoire.

Vous pouvez aussi implémenter un learning rate scheduler, qui par exemple réduit le learning rate au fur et à mesure.

### Warm up learning rate
Un learning rate scheduler assez commun pour les transformers c'est le warm up learning rate. Il consiste à augmenter linéairement le learning rate au début de l'entraînement, puis à le réduire au fur et à mesure:

![transformer learning rate](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/learningrate.png)

Des recherches ont montré que le warm up learning rate permet d'améliorer la performance des transformers.

Il n'y a pas ce scheduler déjà implémenté dans pytorch, donc vous allez devoir l'implémenter vous-même ce custom learning rate scheduler.

Pour implémenter un tel custom learning rate scheduler, il faut créer une classe qui hérite de `torch.optim.lr_scheduler.LambdaLR` et implémenter la fonction `lr_lamba`.
La fonction `lr_lambda` prend en argument l'epoch ou step et retourne le facteur multiplicatif du learning rate.

Par exemple, ici j'ai implémenté un learning rate scheduler qui augmente linéairement le learning rate pendant 1000 steps, puis reste constant:
```python
import torch

class CustomSchedule(torch.optim.lr_scheduler.LambdaLR):
	def __init__(self, optimizer, learning_rate, warm_up_steps, last_epoch=-1):
		self.learning_rate = learning_rate
		self.warm_up_steps = warm_up_steps
		super().__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

	def lr_lambda(self, step):
		if step < self.warm_up_steps:
			return float(step) / float(max(1, self.warm_up_steps))
		return 1.0
```

Puis dans la fonction `configure_optimizers` de votre modèle lightning, vous pouvez utiliser ce scheduler:
```python
def configure_optimizers(self):
    optim = torch.optim.Adam(self.parameters(), lr=0.001)
    scheduler = CustomSchedule(optim, 0.001, 1000)
    scheduler = {
        'scheduler': scheduler,
        'interval': 'step',
        'frequency': 1
    }

    return {
        'optimizer': optim,
        'lr_scheduler': scheduler
    }
```

## Prompts
Maintenant, on va ajouter des prompts à notre modèle. On va donc entraîner notre modèle à écrire des histoires à partir d'un prompt.

### Dataset

In [ ]:
import requests

url = "https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/colab_train_prompts.txt"
r = requests.get(url)
dataset_prompt = r.text

# save file
with open("colab_train_prompts.txt", "w") as f:
	f.write(dataset_prompt)

`dataset_prompt` est une string qui contient les prompts et les histoires associées. Chaque ligne correspond à au prompt associé à chaque histoire de `dataset`.


In [ ]:
print(dataset_prompt.split("\n")[0])  # On affiche la première histoire
print(dataset.split("\n")[0])  # On affiche la première histoire

### Exercice: Preprocessing les données et créer le dataloader

On va mettre les données sous format: "{prompt} <sep> {histoire}", où "\<sep>" est un token qui correspond à une séparation entre le prompt et l'histoire.
Exemple:

In [ ]:
dataset_prompt.split("\n")[0] + " <sep> " + dataset.split("\n")[0]

### Ajouter le token \<sep> à la tokenization BPE de sentencepiece

Il faut ajouter le token "\<sep>" à la tokenization BPE de sentencepiece pour pas que sentencepiece le découpe en sous-mots.
Pour ça, il faut ajouter `--user_defined_symbols=<sep>` dans `sentencepiece_params` et réentraîner la tokenization BPE.
```python
sentencepiece_params = ' '.join([
	'--input={}'.format(input_file),
	'--model_type={}'.format(model_type),
	'--model_prefix={}'.format(model_prefix),
	'--vocab_size={}'.format(max_num_words),
	'--pad_id={}'.format(pad_id),
	'--unk_id={}'.format(unk_id),
	'--bos_id={}'.format(start_id),
	'--eos_id={}'.format(end_id),
	'--user_defined_symbols=<sep>'
])
```

### Créer un mask pour le prompt
Ça ne sert à rien d'entrainer le modèle à prédire le prompt, donc il faut créer un mask pour le prompt, et le multiplier par la loss pour que le modèle ne prédise pas le prompt.
Par exemple: "Le secret de l'immortalité \<sep> histoire ..." alors le mask va être [0, 0, 0, 0, 0, 1, 1, ...] où 0 correspond à chaque token du prompt et 1 correspond à l'histoire.

### Exercice: Création du modèle lightning